In [ ]:
# Import the drive module from the google.colab library to interact with your Google Drive
from google.colab import drive

# Mount your Google Drive to the Colab virtual machine at the directory '/content/drive'
# This requires you to approve permission in a pop-up window
drive.mount('/content/drive')

# Change the current working directory (CWD) to your specific project folder using the magic command %cd
# %cd ensures the directory change persists for all subsequent cells (unlike !cd which only affects one line)
%cd /content/drive/MyDrive/CIS5810 Fa25/project_final

# Optional: Verify the current directory and list files to make sure you are in the right place
import os
print(f"Current Working Directory: {os.getcwd()}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/CIS5810 Fa25/project_final
Current Working Directory: /content/drive/MyDrive/CIS5810 Fa25/project_final


In [ ]:
# --- INSTALLATION COMMANDS (Run these once to set up the Colab environment) ---

# Install OpenCV (headless version for server environments without a display), Pillow for image manipulation, and MediaPipe for facial landmarks
!pip install -q opencv-python-headless pillow mediapipe

# Install Hugging Face libraries: 'diffusers' for Stable Diffusion, 'transformers' for pre-trained models, 'accelerate' for GPU optimization, and 'safetensors' for secure model loading
!pip install -q diffusers transformers accelerate safetensors

# Install the official OpenAI client library to interact with GPT-4 Vision API
!pip install -q openai

# Install Streamlit for building the web interface and pyngrok to expose the local Colab server to the public internet
!pip install -q streamlit pyngrok

# --- IMPORT STATEMENTS (Bringing necessary tools into the script) ---

# Import OpenCV library for computer vision tasks (specifically for Canny edge detection and image array handling)
import cv2

# Import NumPy for numerical operations, essential because images are processed as multi-dimensional arrays (matrices)
import numpy as np

# Import PyTorch, the deep learning framework that powers the Stable Diffusion generation pipeline
import torch

# Import the Image module from Pillow (PIL) to handle image loading, resizing, and saving in a user-friendly format
from PIL import Image

# Import MediaPipe solutions to access the pre-trained Face Mesh model for detecting 478 facial landmarks
import mediapipe as mp

# Import Pyplot from Matplotlib to display images directly within the Google Colab notebook cells for debugging
import matplotlib.pyplot as plt

# Import BytesIO to handle binary streams, allowing us to keep images in memory without saving to disk constantly
from io import BytesIO

# Import Base64 to encode image binary data into text strings, which is required to send images to the OpenAI API
import base64

# Import JSON library to parse the structured text output (dictionaries) returned by GPT-4
import json

# Import OS module to interact with the file system, check if files exist, and handle API keys securely
import os

In [ ]:
# --- SYSTEM CHECK ---

# Check if a GPU (Graphics Processing Unit) is available to PyTorch; this is critical for the generation speed of Stable Diffusion
print(f"GPU available: {torch.cuda.is_available()}")

GPU available: True


In [ ]:
# Import the userdata module from Google Colab to securely access stored secrets (API keys)
from google.colab import userdata

# Import the OpenAI class to create the interface for interacting with GPT models
from openai import OpenAI

# Import the standard 'sys' module to stop execution if necessary (optional, but good for safety)
import sys

# --- STEP 1: RETRIEVE API KEY ---

# Initialize a variable to store the key
api_key = None

try:
    # Attempt to retrieve the 'OPENAI_API_KEY' from Colab's secure "Secrets" store
    # This is the most secure method as the key is not visible in the code
    api_key = userdata.get('OPENAI_API_KEY')
    print("Success: API Key found in Colab Secrets.")

except (userdata.SecretNotFoundError, userdata.NotebookAccessError):
    # If the secret doesn't exist or permissions are missing, catch the specific error
    print("Notice: 'OPENAI_API_KEY' not found in Secrets. Switching to manual input.")
    pass # Continue to the manual input block below

# --- STEP 2: MANUAL FALLBACK & VALIDATION ---

# If the key is still None (because Secrets failed) or empty, ask the user manually
if not api_key:
    # Prompt the user to paste the key; usage of .strip() removes accidental leading/trailing spaces
    api_key = input("Please enter your OpenAI API Key: ").strip()

# --- STEP 3: FINAL VALIDATION ---

# specific check: ensure the user didn't just press "Enter" leaving the key empty
if not api_key:
    # Print a critical error message in red (using ANSI escape codes) for visibility
    print("\033[91mError: No API Key provided. The script cannot proceed.\033[0m")
    # Stop the script execution immediately to prevent crashing later
    sys.exit("Script stopped due to missing API Key.")

# --- STEP 4: INITIALIZE CLIENT ---

# Initialize the OpenAI client with the validated key
# This client object will be used for all subsequent calls (GPT-4, DALL-E, etc.)
client = OpenAI(api_key=api_key)

# Confirm successful configuration to the user
print("OpenAI Client successfully configured and ready.")

Success: API Key found in Colab Secrets.
OpenAI Client successfully configured and ready.


In [ ]:
1 + 3

4

In [ ]:
# Initialize MediaPipe Face Mesh globally to prevent reloading the heavy model every time the function is called
mp_face_mesh = mp.solutions.face_mesh

def get_face_landmarks(image_path):
    """
    Detects face landmarks using MediaPipe and loads the image once.
    """
    # Load the image from the disk using OpenCV (loads in BGR format)
    image = cv2.imread(image_path)

    # Validation check: if the image path is wrong or file is corrupted, 'image' will be None
    if image is None:
        # Raise an error to stop execution and inform the user
        raise FileNotFoundError(f"Image not found at: {image_path}")

    # Convert the image from BGR (OpenCV standard) to RGB (MediaPipe requirement)
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Open the FaceMesh model with specific settings using a context manager ('with') to handle memory automatically
    with mp_face_mesh.FaceMesh(
        static_image_mode=True,       # We are processing a single photo, not a video stream
        max_num_faces=1,              # Limit detection to the most prominent face only
        refine_landmarks=True,        # Enable detailed attention to eyes and lips (478 points)
        min_detection_confidence=0.5  # Set confidence threshold to 50%
    ) as face_mesh:

        # Run the inference to detect facial landmarks on the RGB image
        results = face_mesh.process(rgb_image)

    # Check if any faces were actually detected
    if not results.multi_face_landmarks:
        # If the list is empty, raise an error
        raise ValueError("No face detected. Please try a clearer photo.")

    # Return the landmarks of the first face found [0], and the original BGR image array for later use
    return results.multi_face_landmarks[0], image


def create_feature_canny_map(image_bgr, feature_name, landmarks):
    """
    Generates a Canny edge map isolated to a specific facial feature.
    Handles 'split' features (like left/right eyes) separately to avoid incorrect masking.
    """
    # Extract height (h) and width (w) from the image array shape
    h, w, _ = image_bgr.shape

    # --- 1. FEATURE DEFINITIONS ---
    # We use a LIST OF LISTS structure.
    # Single regions (like mouth) have one inner list: [[points]].
    # Split regions (like eyes) have two inner lists: [[left_points], [right_points]].

    feature_indices = {
        # EYES: Split into Left and Right so the mask doesn't cover the nose bridge
        "eyes": [
            [33, 246, 161, 160, 159, 158, 157, 173, 133, 155, 154, 153, 145, 144, 163, 7], # Left Eye Loop
            [362, 398, 384, 385, 386, 387, 388, 466, 263, 249, 390, 373, 374, 380, 381, 382] # Right Eye Loop
        ],

        # MOUTH: Single region containing lips and mouth opening
        "mouth": [[61, 146, 91, 181, 84, 17, 314, 405, 321, 375, 291, 308, 324, 318, 402, 317,
                   14, 87, 178, 88, 95, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291]],

        # JAWLINE: Single region describing the lower face curve
        "jawline": [[234, 93, 132, 58, 172, 136, 150, 149, 176, 148, 152, 377, 400, 378, 379,
                     365, 397, 288, 361, 323, 454, 356]],

        # NOSE: Single region for nose tip and nostrils
        "nose": [[1, 2, 98, 327, 168, 6, 197, 195, 5, 4, 45, 275, 440, 220, 243]],

        # EYEBROWS: Split Left and Right so we don't draw a "unibrow" mask across the forehead
        "eyebrows": [
            [70, 63, 105, 66, 107, 55, 65, 52, 53, 46],  # Left Brow
            [336, 296, 334, 293, 300, 276, 283, 282, 295, 285] # Right Brow
        ],

        # CHEEKS: Split Left and Right so we don't mask out the nose/mouth in the center
        "cheeks": [
            [116, 123, 147, 213, 192, 214], # Left Cheek area
            [345, 352, 376, 433, 416, 434]  # Right Cheek area
        ]
    }

    # --- 2. GLOBAL CANNY GENERATION ---

    # Convert the BGR image to Grayscale (0-255) for edge detection
    gray = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2GRAY)

    # Calculate the median pixel intensity of the image
    v = np.median(gray)

    # Set a tightness parameter (sigma) for the dynamic thresholding
    sigma = 0.33

    # Calculate lower threshold (max ensures it's not negative)
    lower = int(max(0, (1.0 - sigma) * v))

    # Calculate upper threshold (min ensures it doesn't exceed 255)
    upper = int(min(255, (1.0 + sigma) * v))

    # Apply Canny algorithm to find all edges in the image using calculated thresholds
    canny_full = cv2.Canny(gray, lower, upper)

    # --- 3. MASK GENERATION ---

    # Create a black canvas (all zeros) with the same size as the grayscale image
    mask = np.zeros_like(gray)

    # Normalize the input feature name to lowercase for dictionary lookup
    fname = feature_name.lower()

    # Check if the requested feature exists in our definitions
    if fname not in feature_indices:
        print(f"Warning: {fname} not found. Returning full edges.")
        # If unknown, return the full edge map without masking
        return Image.fromarray(canny_full)

    # Retrieve the list of regions for the requested feature (e.g., [[left_pts], [right_pts]])
    regions = feature_indices[fname]

    # Loop through each sub-region separately (CRITICAL FIX for split features)
    for indices in regions:
        points = []

        # Convert landmark indices to pixel coordinates
        for idx in indices:
            pt = landmarks.landmark[idx] # Get normalized point (0.0-1.0)
            x = int(pt.x * w)            # Scale to image width
            y = int(pt.y * h)            # Scale to image height
            points.append((x, y))        # Add to list

        # Convert list of tuples to a NumPy array for OpenCV drawing functions
        points_np = np.array(points, dtype=np.int32)

        # Draw the white region on the black mask
        if fname == "jawline":
             # For jawline, we draw a thick open line (polyline) because it's not a loop
             cv2.polylines(mask, [points_np], isClosed=False, color=255, thickness=40)
        else:
             # For all other features (eyes, nose, etc.), we draw a filled closed polygon
             # convexHull wraps the points in the smallest possible "rubber band" shape
             hull = cv2.convexHull(points_np)

             # Fill the hull with white (255)
             cv2.fillConvexPoly(mask, hull, 255)

    # --- 4. MASK REFINEMENT ---

    # Create a 15x15 kernel of 1s for dilation
    kernel = np.ones((15,15), np.uint8)

    # Dilate (expand) the white mask slightly to include edges bordering the feature
    mask = cv2.dilate(mask, kernel, iterations=1)

    # --- 5. FINAL COMBINATION ---

    # Perform Bitwise AND: Only keep pixels from 'canny_full' where 'mask' is white
    feature_canny = cv2.bitwise_and(canny_full, canny_full, mask=mask)

    # Convert the resulting NumPy array to a PIL Image object (required for the Diffusion pipeline)
    return Image.fromarray(feature_canny)

In [ ]:
def analyze_face_with_gpt4(image_path):
    """
    Sends image to GPT-4 Vision to select 3 animals from a FIXED POOL based on facial features.
    """

    # --- 1. IMAGE ENCODING ---
    # Open the image file in binary read mode
    with open(image_path, "rb") as image_file:
        # Encode the image to base64 string so it can be sent via JSON API
        base64_image = base64.b64encode(image_file.read()).decode('utf-8')

    # --- 2. PROMPT ENGINEERING WITH FIXED POOL ---

    prompt_text = """
    You are an expert visual-anthropology model.
    Analyze the provided human portrait.

    TASK:
    Select EXACTLY THREE animals from the fixed pool below that best resemble the person's visual traits.

    === ALLOWED ANIMAL POOL (Choose 3) ===
    ["Bear", "Beaver", "Cat", "Deer", "Eagle", "Elephant", "Fox", "Hamster",
     "Horse", "Koala", "Lamb", "Lion", "Meerkat", "Owl", "Panda", "Puppy",
     "Racoon", "Sloth", "Tiger", "Wolf"]

    === ALLOWED FEATURE POOL (Choose 3 distinct) ===
    ["eyes", "jawline", "mouth", "nose", "eyebrows", "cheeks"]

    === RULES ===
    1. STRICT RESTRICTION: Do NOT use any animal outside the Allowed Pool.
    2. STRICT RESTRICTION: The "feature" must be exactly one of the Allowed Features.
    3. DIVERSITY: Select 3 DIFFERENT animals and 3 DIFFERENT features.
       (e.g. Do not use 'eyes' for all three animals).
    4. OUTPUT FORMAT: Return ONLY raw JSON.

    === JSON EXAMPLE ===
    {
      "animals": [
        {"animal": "Owl", "trait": "wise, observant", "feature": "eyes"},
        {"animal": "Fox", "trait": "clever, alert", "feature": "nose"},
        {"animal": "Deer", "trait": "gentle, soft", "feature": "jawline"}
      ]
    }
    """

    # --- 3. API CALL ---
    try:
        response = client.chat.completions.create(
            model="gpt-4o", # Recommended model for best instruction following
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt_text},
                        {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}},
                    ],
                }
            ],
            max_tokens=300,
            response_format={"type": "json_object"} # Enforce JSON structure
        )

        # --- 4. PARSING RESPONSE ---
        content = response.choices[0].message.content
        result_json = json.loads(content)

        # Validation: Check if returned animals are actually in our pool (Optional safety check)
        valid_pool = ["bear", "beaver", "cat", "deer", "eagle", "elephant", "fox", "hamster",
                      "horse", "koala", "lamb", "lion", "meerkat", "owl", "panda", "puppy",
                      "racoon", "sloth", "tiger", "wolf"]

        for item in result_json.get("animals", []):
            if item['animal'].lower() not in valid_pool:
                print(f"Warning: GPT returned {item['animal']} which is not in pool. Replacing with 'Cat'.")
                item['animal'] = "Cat" # Fallback mechanism

        return result_json

    except Exception as e:
        print(f"Error during GPT-4 analysis: {e}")
        # Return a safe fallback from the pool
        return {
            "animals": [
                {"animal": "Cat", "trait": "default", "feature": "eyes"},
                {"animal": "Puppy", "trait": "default", "feature": "mouth"},
                {"animal": "Eagle", "trait": "default", "feature": "nose"}
            ]
        }

In [ ]:
import torch
from diffusers import StableDiffusionXLControlNetPipeline, ControlNetModel, AutoencoderKL, DPMSolverMultistepScheduler

def load_generation_pipeline():
    """
    Loads the SDXL ControlNet pipeline with memory optimizations for Colab T4 GPU.
    """
    print("Loading ControlNet Model...")
    # 1. Load ControlNet for Canny Edge (SDXL version)
    # This model translates the white-on-black edge maps into guidance for the AI
    controlnet = ControlNetModel.from_pretrained(
        "diffusers/controlnet-canny-sdxl-1.0",
        torch_dtype=torch.float16,  # Use half-precision to save VRAM
        use_safetensors=True        # Faster and safer model loading format
    )

    print("Loading VAE...")
    # 2. Load VAE (Variational Autoencoder)
    # We use a specific "fp16-fix" version. Standard SDXL VAE often produces black images or "NaN" errors
    # when running in float16 precision. This fixed version prevents that.
    vae = AutoencoderKL.from_pretrained(
        "madebyollin/sdxl-vae-fp16-fix",
        torch_dtype=torch.float16,
        use_safetensors=True
    )

    print("Loading Main SDXL Pipeline...")
    # 3. Load the Main Pipeline (Stable Diffusion XL Base 1.0)
    pipe = StableDiffusionXLControlNetPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-base-1.0",
        controlnet=controlnet,
        vae=vae,
        torch_dtype=torch.float16,
        use_safetensors=True,
        variant="fp16" # Loads the smaller fp16 weights directly to save download time
    )

    # 4. OPTIMIZATION: Scheduler Upgrade
    # Switch to DPM++ 2M Karras. It produces better details in fewer steps (25-30 steps)
    # compared to the default scheduler (which often needs 50+).
    pipe.scheduler = DPMSolverMultistepScheduler.from_config(
        pipe.scheduler.config,
        use_karras_sigmas=True,
        algorithm_type="dpmsolver++"
    )

    # 5. OPTIMIZATION: Memory Management for Colab T4 (16GB VRAM)
    # "cpu_offload" keeps only the active component (e.g., UNet) in GPU VRAM,
    # and moves others (Text Encoder, VAE) to CPU RAM when not in use.
    # This is CRITICAL to avoid "RuntimeError: CUDA out of memory".
    pipe.enable_model_cpu_offload()

    # Optional: Enable VAE slicing to save even more memory during the final image decoding phase
    pipe.enable_vae_slicing()

    # VAE Tiling: Decodes in tiles (The most important fix for 1024x1024 on T4)
    pipe.enable_vae_tiling()

    return pipe

# Initialize pipeline (Run this once, it takes about 1-2 minutes to download ~6GB of models)
pipe = load_generation_pipeline()
print("✅ Stable Diffusion XL ControlNet Pipeline Loaded Successfully.")

Loading ControlNet Model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading VAE...
Loading Main SDXL Pipeline...


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


✅ Stable Diffusion XL ControlNet Pipeline Loaded Successfully.


In [ ]:
import gc # Garbage Collector interface

def process_animorphic_triptych(image_path):
    print(f"\n🚀 Starting processing for: {image_path}")

    # --- PHASE 1: ANALYSIS ---
    print("Step 1/4: Extracting Face Landmarks...")
    landmarks, image_bgr = get_face_landmarks(image_path)

    print("Step 2/4: Consulting GPT-4 Visual Anthropologist...")
    analysis = analyze_face_with_gpt4(image_path)

    if "animals" not in analysis:
        print("Error: GPT response invalid.")
        return

    animals_data = analysis["animals"]
    print(f"GPT Output: {json.dumps(animals_data, indent=2)}")

    # --- PHASE 2: SYNTHESIS LOOP ---
    generated_results = []
    print(f"Step 3/4: Generating 3 Animal Portraits...")

    for i, item in enumerate(animals_data):
        animal = item['animal']
        trait = item['trait']
        feature = item['feature']

        print(f"  > Processing {i+1}/3: {animal} (focus: {feature})...")

        # 1. Prepare Canny Map
        canny_map = create_feature_canny_map(image_bgr, feature, landmarks)

        # 2. Prompt
        prompt = (
            f"A majestic {trait} {animal} portrait, close-up, highly detailed, "
            f"cinematic lighting, 8k resolution, photorealistic texture, "
            f"expressive {feature}, artstation trending, masterpiece."
        )
        negative_prompt = "human face, skin, cartoon, anime, sketch, blurry, low quality, distorted, deformed, text, watermark"

        # 3. Generate Image
        try:
            image = pipe(
                prompt,
                negative_prompt=negative_prompt,
                image=canny_map,
                controlnet_conditioning_scale=0.65,
                num_inference_steps=30,
                guidance_scale=7.5
            ).images[0]

            generated_results.append({
                "animal": animal,
                "feature": feature,
                "trait": trait,
                "canny": canny_map,
                "image": image
            })

        except Exception as e:
            print(f"Error generating {animal}: {e}")
            continue

        # --- CRITICAL FIX: FORCE MEMORY CLEANUP ---
        # After each generation, force clear the GPU cache to make room for the next one
        print("    ...Cleaning GPU memory...")
        torch.cuda.empty_cache()
        gc.collect()

    # --- PHASE 3: VISUALIZATION ---
    print("Step 4/4: Visualization...")

    if not generated_results:
        print("No images were generated successfully.")
        return

    # Dynamic figure size based on how many images succeeded
    count = len(generated_results) + 1
    fig, axes = plt.subplots(1, count, figsize=(5 * count, 6))

    # Handle single axis case if only 1 image generated
    if count == 1: axes = [axes]

    # Show Original
    axes[0].imshow(cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB))
    axes[0].set_title("Original")
    axes[0].axis("off")

    # Show Generated
    for i, res in enumerate(generated_results):
        axes[i+1].imshow(res['image'])
        axes[i+1].set_title(f"{res['animal']}\n({res['feature']})")
        axes[i+1].axis("off")

    plt.tight_layout()
    plt.show()

    return generated_results

# --- RUN AGAIN ---
input_image = "test_face.jpg"
if os.path.exists(input_image):
    results = process_animorphic_triptych(input_image)
else:
    print(f"Please check if {input_image} exists.")

NameError: name 'os' is not defined

In [ ]:
def process_animorphic_triptych(image_path):
    # 1. Resize/Preprocess Image (Ensure it's 1024x1024 for SDXL best results)
    original_img = Image.open(image_path).convert("RGB")
    original_img = original_img.resize((1024, 1024))
    original_img.save("aligned_input.jpg")

    # 2. Get Landmarks
    print("Extracting landmarks...")
    landmarks, shape = get_face_landmarks("aligned_input.jpg")

    # 3. Get GPT-4 Analysis
    print("Analyzing face with GPT-4...")
    analysis = analyze_face_with_gpt4("aligned_input.jpg")
    print("GPT Analysis:", json.dumps(analysis, indent=2))

    results = []

    # 4. Loop through the 3 archetypes
    for item in analysis['animals']:
        animal = item['animal']
        trait = item['trait']
        feature = item['feature']

        print(f"Generating {animal} based on {feature}...")

        # A. Create the Feature-Specific Canny Map
        canny_image = create_feature_canny_map("aligned_input.jpg", feature, landmarks, shape)

        # B. Define the Prompt
        # We use a structured prompt for SDXL
        prompt = f"A highly detailed, artistic portrait of a {animal}, {trait}, cinematic lighting, 8k resolution, photorealistic, intricate textures."
        negative_prompt = "cartoon, drawing, anime, blurry, low quality, distorted, human face"

        # C. Generate Image
        # controlnet_conditioning_scale determines how strictly it follows the lines
        # For eyes/mouth, we want strict adherence (0.7-0.9). For jawline, maybe less (0.5-0.7).
        scale = 0.8 if feature != "jawline" else 0.6

        gen_image = pipe(
            prompt,
            negative_prompt=negative_prompt,
            image=canny_image,
            controlnet_conditioning_scale=scale,
            num_inference_steps=30,
            guidance_scale=7.5
        ).images[0]

        results.append({
            "animal": animal,
            "feature": feature,
            "canny_map": canny_image,
            "result": gen_image
        })

    return original_img, results

# --- RUNNING THE TEST ---
# Upload an image to Colab first and name it 'test_face.jpg'
# OR uncomment below to upload interactively:
# from google.colab import files
# uploaded = files.upload()
# image_path = list(uploaded.keys())[0]

# For demo, assuming you have 'test_face.jpg'
if os.path.exists('test_face.jpg'):
    original, output_data = process_animorphic_triptych('test_face.jpg')

    # Display using Matplotlib for immediate check
    plt.figure(figsize=(15, 5))

    # Show Original
    plt.subplot(1, 4, 1)
    plt.imshow(original)
    plt.title("Original")
    plt.axis('off')

    # Show Generations
    for i, data in enumerate(output_data):
        plt.subplot(1, 4, i+2)
        plt.imshow(data['result'])
        plt.title(f"{data['animal']}\n({data['feature']})")
        plt.axis('off')

    plt.show()
else:
    print("Please upload an image named 'test_face.jpg' to run the test.")

In [ ]:
%%writefile app.py
import streamlit as st
import cv2
import numpy as np
from PIL import Image
import mediapipe as mp
import base64
import json
import torch
from diffusers import StableDiffusionXLControlNetPipeline, ControlNetModel, AutoencoderKL

# --- PAGE CONFIG ---
st.set_page_config(page_title="Animorphic Triptych", layout="wide")

st.title("🦁 Animorphic Triptych")
st.markdown("### From Human Portrait to Animal Archetypes")

# --- SETUP (Copy core functions here or import them if structured as a module) ---
# For simplicity in this single-file demo, we assume the pipeline is loaded
# inside the app or cached. Due to Colab RAM limits, reloading heavy models
# inside Streamlit subprocess might crash if not handled carefully.
# IN COLAB: It is better to use the notebook for processing and Streamlit just for display
# But for a standalone app, you would load models here using @st.cache_resource

@st.cache_resource
def load_models():
    # Only load if not already in memory to save time
    controlnet = ControlNetModel.from_pretrained(
        "diffusers/controlnet-canny-sdxl-1.0", torch_dtype=torch.float16
    )
    vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16)
    pipe = StableDiffusionXLControlNetPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-base-1.0",
        controlnet=controlnet, vae=vae, torch_dtype=torch.float16
    )
    pipe.enable_model_cpu_offload()
    return pipe

# Placeholder for the processing logic (users should copy the functions from Step 3 & 4 here)
# ... [Insert get_face_landmarks, create_feature_canny_map, analyze_face_with_gpt4 here] ...

# --- UI LOGIC ---
uploaded_file = st.file_uploader("Upload a Portrait", type=["jpg", "png", "jpeg"])

if uploaded_file is not None:
    # Display Original
    image = Image.open(uploaded_file).convert("RGB")
    st.image(image, caption="Original Portrait", width=300)

    if st.button("Generate Triptych"):
        with st.spinner("Loading AI Models..."):
            pipe = load_models()

        # Save temp file for OpenCV
        image.save("temp_input.jpg")

        # 1. GPT Analysis
        with st.spinner("Analyzing Personality & Features..."):
            # Dummy mock for UI testing if API key not set, else real call
            # analysis = analyze_face_with_gpt4("temp_input.jpg")
            st.warning("Please ensure API Key and functions are fully integrated in app.py source.")

        # ... logic to generate and display columns ...
        col1, col2, col3 = st.columns(3)

        with col1:
            st.markdown("#### Animal 1")
            # st.image(result1)

        with col2:
            st.markdown("#### Animal 2")

        with col3:
            st.markdown("#### Animal 3")

st.info("Note: This is a skeletal UI. To run fully in Colab, copy all helper functions into app.py")

In [ ]:
# Run Streamlit in the background
!streamlit run app.py &>/dev/null&

# Expose via LocalTunnel
import urllib
print("Password/IP for LocalTunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
!npx localtunnel --port 8501